In [6]:
import numpy as np

def preprocess(text):
  text = text.lower()
  text = text.replace('.', ' .')
  
  words = text.split(' ')
  
  word_to_id = {}
  id_to_word = {}
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word
      
  corpus = np.array([word_to_id[w] for w in words])
  return corpus, word_to_id, id_to_word

In [8]:
text = "You say goodby and I say hello"
preprocess(text)

(array([0, 1, 2, 3, 4, 1, 5]),
 {'you': 0, 'say': 1, 'goodby': 2, 'and': 3, 'i': 4, 'hello': 5},
 {0: 'you', 1: 'say', 2: 'goodby', 3: 'and', 4: 'i', 5: 'hello'})

분산표현 (distributional representation)
- 단어의 의미를 정확하게 파악할 수 있는 벡터 표현
- 단어를 고정 길이의 밀집벡터로 표현

분포가설 (distributional hypothesis)
- 단어의 의미는 주변 단어에 의해 형성된다.
- 단어 자체에는 의미가 없고, 그 단어가 사용된 '맥락'이 의미를 형성한다.

통계기반 기법 
- 주위에 어떤 단어가 몇 벙니나 등장하는지를 집계하는 방법

동시발생 행렬 (co-occurence matrix)
- 모든 단어에 대해 동시 발생하는 단어를 표에 정리한 것

In [9]:
def create_co_matrix(corpus, vocab_size, window_size=1):
  corpus_size = len(corpus)
  co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
  
  for idx, word_id in enumerate(corpus):
    for i in range(1, window_size + 1):
      left_idx = idx - 1
      right_idx = idx + 1
      
      if left_idx >= 0:
        left_word_id = corpus[left_idx]
        co_matrix[word_id, left_word_id] += 1
        
      if right_idx < corpus_size:
        right_word_id = corpus[right_idx]
        co_matrix[word_id, right_word_id] += 1
        
  return co_matrix

In [10]:
def cos_similarity(x, y, eps=1e-8):
  nx = x / np.sqrt(np.sum(x ** 2) + eps)
  ny = y / np.sqrt(np.sum(y ** 2) + eps)
  return np.dot(nx, ny)

In [12]:
import sys
sys.path.append('../../modules/Part2/')
from common.utils import preprocess, create_co_matrix, cos_similarity

text = "You say goodbye and I say hello"
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]

print(cos_similarity(c0, c1))

0.7071067758832467


In [ ]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
  if query not in word_to_id:
    print('{query}를 찾을 수 없습니다.')
    return
  
  print(f'\n [{query}]')
  query_id = word_to_id[query]
  query_vec = word_matrix[query_id]
  
  vocab_size = len(id_to_word)
  similarity = np.zeros(vocab_size)
  for i in range(vocab_size):
    similarity[i] = cos_similarity(word_matrix[i], query_vec)